__[Read a pdf version of this post here](https://github.com/alisaaalehi/convolution_as_multiplication/blob/master/ConvAsMulExplained.pdf)__


# What is the purpose?

Instead of using `for-loops` to perform 2D convolution on images (or any other 2D matrices) we can convert the filter to a `Toeplitz matrix` and image to a vector and do the convolution just by one `matrix multiplication` (and of course some post-processing on the result of this multiplication to get the final result)

# Why do we do that?
There are many efficient matrix multiplication algorithms, so using them we can have an efficient implementation of convolution operation.

# What is in this document?
Mathematical and algorithmic explanation of this process. I will put a naive Python implementation of this algorithm to make it more clear.<br>
Let's start with some definition and basic operation:

## What is a Toeplitz matrix?
Toeplitz matrix is a matrix in which each values along the main diagonal and sub diagonals are constant. Matrix $G$ is an example:

$$\begin{pmatrix}
	2 & -1 & 0 & \cdots & \cdots & \cdots & \cdots & 1\\
	5 & 2 & -1 & 0 & & & & \vdots\\
	-8 & 5 & 2 & -1 & \ddots & & & \vdots\\
	\vdots & -8 & \ddots & \ddots & \ddots & \ddots & & \vdots\\
	\vdots & & \ddots & \ddots & \ddots & \ddots & 0 & \vdots\\
	\vdots & & & \ddots & 5 & 2 & -1 & 0\\
	\vdots & & & & -8 & 5 & 2 & -1\\
	1 & \cdots & \cdots  & \cdots & \cdots & -8 & 5 & 2\\
	\end{pmatrix}$$
    
In a $N \times N$ matrix, its elements are determined by a ($2N -1$) length sequence
	 $$\{t_n | -(N-1) \le n \le (N-1)\}$$

So given a sequence $t_n$ we can create a Toeplitz matrix by following these steps:    
 - put the sequence in the first column of the matrix.
 - shift it and put it in the next column. When shifting, the last element disappears and a new element of the sequence appears. If there is no such an element, put zero in that location. 

specifically: $T(m,n)=t_{m-n}$

$$\begin{pmatrix}
	t_0 & t_{-1} & t_{-2} & \cdots & \cdots & \cdots & \cdots & t_{-(N-1)}\\
	t_1 & t_0 & t_{-1} & t_{-2} & & & & \vdots\\
	t_2 & t_1 & t_0 & t_{-1} & \ddots & & & \vdots\\
	\vdots & t_2 & \ddots & \ddots & \ddots & \ddots & & \vdots\\
	\vdots & & \ddots & \ddots & \ddots & \ddots & t_{-2} & \vdots\\
	\vdots & & & \ddots & t_1 & t_0 & t_{-1} & t_{-2}\\
	\vdots & & & & t_2 & t_1 & t_0 & t_{-1}\\
	t_{(N-1)} & \cdots & \cdots  & \cdots & \cdots & t_2 & t_1 & t_0\\
	\end{pmatrix}$$

Be aware that when we are working with sequences that are defined just for $n \ge 0$ values for $t_n$ when $n \le 0$ should be considered as $0$. For example $4 \times 4$ Toeplitz matrix for the sequence $f[n]=[1,2,3]$ will be:

$$\begin{pmatrix}
	1 & 0 & 0 & 0\\
	2 & 1 & 0 & 0\\
	3 & 2 & 1 & 0\\
	0 & 3 & 2 & 1 \\
	\end{pmatrix}$$
    
<a id="doubly_blocked"></a>   
## One more definition: Doubly Blocked Toeplitz mtrix
In the matrix $A$, all $A_{ij}$ are matrices. If the structure of $A$, with respects to its sub-matrices is Toeplitz, then matrix $A$ is called \textbf{Block-Toeplitz}. 

$$\begin{pmatrix}
	A_{11} & A_{12} & \cdots & A_{1N}\\
	A_{21} & A_{22} & \cdots & A_{2N}\\
	\vdots & \vdots & \vdots & \vdots \\
	A_{M1} & A_{M2} & \cdots & A_{MN}\\
	\end{pmatrix}$$
	
If each individual $A{ij}$ also is a Toeplitz matrix then $A$ is called \textbf{Doubly Blocked Toeplitz}
	
# Is this Convolution or Cross Correlation?
Most of the time, the word *convolution* in the deep learning literature is used instead of *cross-correlation*, but here I am explaining the process for convolution as is known in the signal processing community. Simply, for convolution we need to flip the filter (kernel) in both vertical and horizontal directions, but for cross-correlation we don't. 
	
The method explained here performs the convolution (not correlation). Because of the way it is implemented here, there is no need to flip the filter, but if you are doing an example by hand and want to compare the results with the implemented method, remember to consider the flipping step in your calculation.

# Step by Step
Let's explain the algorithm step by step using an example. Codes are written in python and the numpy library is used all over the code.
	
**Note:** Remember that convolution is a commutative operation, so it does not change the output if we switch the inputs for this operation. For simplicity, I will be calling one of the inputs *input* or `I` and the other *filter* or `F`
	
## Input and Filter
Input matrix that the filter will be convolved with it, is:

$$I = 
	\begin{bmatrix}
	1 & 2 & 3 \\
	4 & 5 & 6 \\
	\end{bmatrix}$$
	
And let the filter be:

$$F = 
	\begin{bmatrix}
	10 & 20 \\
	30 & 40 \\
	\end{bmatrix}$$
   

In [21]:
import numpy as np

# input signal
I = np.array([[1, 2, 3], [4, 5, 6]])
print('I: ', I.shape)
print(I)

 # filter 
F = np.array([[10, 20], [30, 40]])
print('F: ',F.shape)
print(F)

('I: ', (2, 3))
[[1 2 3]
 [4 5 6]]
('F: ', (2, 2))
[[10 20]
 [30 40]]


<a id="output-size"></a>
## Calculate the final output size
If the input signal is $m_1 \times n_1$ and filter is $m_2 \times n_2$ the size of the convolution will be 

$$(m_1 + m_2 -1) \times (n_1 + n_2 -1)$$

This is the size of full discrete linear convolution. One might just use some part of the output based on the application. For example in deep learning literature you can use "valid" or "same" as your padding mode. In these case just parts of the full output is used.

Proper zero padding should be done to get the correct output. Zero padding is the next step.  

In [22]:
# number columns and rows of the input 
I_row_num, I_col_num = I.shape 

# number of columns and rows of the filter
F_row_num, F_col_num = F.shape

#  calculate the output dimensions
output_row_num = I_row_num + F_row_num - 1
output_col_num = I_col_num + F_col_num - 1
print('output dimension:', output_row_num, output_col_num)


('output dimension:', 3, 4)


## Zero-pad the filter matrix
The next step is to zero pad the filter to make it the same size as the output. Zeros should be added to the top and right sides of the filter.

<img src="images/3.png" alt="Zero padding" title="Zero padded F" />

In [23]:
# zero pad the filter
F_zero_padded = np.pad(F, ((output_row_num - F_row_num, 0),
                           (0, output_col_num - F_col_num)),
                        'constant', constant_values=0)
print('F_zero_padded: ', F_zero_padded)

('F_zero_padded: ', array([[ 0,  0,  0,  0],
       [10, 20,  0,  0],
       [30, 40,  0,  0]]))


## Toeplitz matrix for each row of the zero-padded filter
For each row of the zero-padded filter `F_zero_padded` create a Toeplitz matrix and store them in a list. Matrix created using the last row goes to the first cell of this list.

<img src="images/4.png" alt="Toeplitz matrix for each row of the zero-padded filter" title="Toeplitz matrix for each row of the zero-padded filter" />

**Why these matrices have three columns? Why not two or 5? What is the rule here?**
The important point is that the number of columns of these generated Toeplitz matrices should be same as the number of columns of the input (I) matrix.

In the code, I am using `toeplitz()` function from `scipy.linalg library`. One row of the $F$ is passed to this function and the function puts it as the first column of the its output matrix. Then as it is explained before, this vector should be shifted down and be putted in the second column. For this function, in addition to the first column, we need to define the first row, otherwise, the output of the function would be different than what we expect here. The first element of this first row is same as the first element of the first column, and the rest of the elements should be set to zero. (I know that it doesn't make sense :D read it twice and look at the code)

In [24]:
from scipy.linalg import toeplitz

# use each row of the zero-padded F to creat a toeplitz matrix. 
#  Number of columns in this matrices are same as numbe of columns of input signal
toeplitz_list = []
for i in range(F_zero_padded.shape[0]-1, -1, -1): # iterate from last row to the first row
    c = F_zero_padded[i, :] # i th row of the F 
    r = np.r_[c[0], np.zeros(I_col_num-1)] # first row for the toeplitz fuction should be defined otherwise
                                                        # the result is wrong
    toeplitz_m = toeplitz(c,r) # this function is in scipy.linalg library
    toeplitz_list.append(toeplitz_m)
    print('F '+ str(i)+'\n', toeplitz_m)

('F 2\n', array([[30.,  0.,  0.],
       [40., 30.,  0.],
       [ 0., 40., 30.],
       [ 0.,  0., 40.]]))
('F 1\n', array([[10.,  0.,  0.],
       [20., 10.,  0.],
       [ 0., 20., 10.],
       [ 0.,  0., 20.]]))
('F 0\n', array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]))


## Create doubly blocked toeplitz matrix
Now all these small toeplitz matrices should be arranged in a big doubly blocked toepltiz matrix [as described in this section](#doubly_blocked).

In this example $F_0, F_1, F_2$ are corresponding toeplitz matrices for each row of the filter. They should be filled in the doubly blocked toeplitz matrix in this way:

$$doubly \ blocked = 
	\begin{bmatrix}
	F_0 & 0 \\
	F_1 & F_0 \\
	F_2 & F_1 \\
	\end{bmatrix}$$
    
Number of columns in this symbolic matrix should be same as the number of rows in the input signal I.\\
	The following code stores the indexes of $F_0, F_1, F_2$ in this format. This will be used to fill out the doubly blocked toepltiz matrix later.

In [25]:
# doubly blocked toeplitz indices: 
#  this matrix defines which toeplitz matrix from toeplitz_list goes to which part of the doubly blocked
c = range(1, F_zero_padded.shape[0]+1)
r = np.r_[c[0], np.zeros(I_row_num-1, dtype=int)]
doubly_indices = toeplitz(c, r)
print('doubly indices \n', doubly_indices)

('doubly indices \n', array([[1, 0],
       [2, 1],
       [3, 2]]))


Now let's fill in the doubly blocked toepltiz matrix. Following code does this part:

In [26]:
## creat doubly blocked matrix with zero values
toeplitz_shape = toeplitz_list[0].shape # shape of one toeplitz matrix
h = toeplitz_shape[0]*doubly_indices.shape[0]
w = toeplitz_shape[1]*doubly_indices.shape[1]
doubly_blocked_shape = [h, w]
doubly_blocked = np.zeros(doubly_blocked_shape)

# tile toeplitz matrices for each row in the doubly blocked matrix
b_h, b_w = toeplitz_shape # hight and withs of each block
for i in range(doubly_indices.shape[0]):
    for j in range(doubly_indices.shape[1]):
        start_i = i * b_h
        start_j = j * b_w
        end_i = start_i + b_h
        end_j = start_j + b_w
        doubly_blocked[start_i: end_i, start_j:end_j] = toeplitz_list[doubly_indices[i,j]-1]

print('doubly_blocked: ', doubly_blocked)

('doubly_blocked: ', array([[30.,  0.,  0.,  0.,  0.,  0.],
       [40., 30.,  0.,  0.,  0.,  0.],
       [ 0., 40., 30.,  0.,  0.,  0.],
       [ 0.,  0., 40.,  0.,  0.,  0.],
       [10.,  0.,  0., 30.,  0.,  0.],
       [20., 10.,  0., 40., 30.,  0.],
       [ 0., 20., 10.,  0., 40., 30.],
       [ 0.,  0., 20.,  0.,  0., 40.],
       [ 0.,  0.,  0., 10.,  0.,  0.],
       [ 0.,  0.,  0., 20., 10.,  0.],
       [ 0.,  0.,  0.,  0., 20., 10.],
       [ 0.,  0.,  0.,  0.,  0., 20.]]))


For this example the result will be the following matrix. I've colored parts of the matrix that is related to each of the small toeplitz matrices.

<img src="images/6.png" alt="doubly blocked toeplitz" title="doubly blocked toeplitz matrix">

## Convert the input matrix to a vector
Now that the filter has converted to a doubly blocked Toeplitz matrix, we just need to convert the input signal to a vector and multiply them. 

All the rows of the input should be transposed to a column vector and stacked on top of each other. The last row goes first!

<img src="images/7.png" alt="vectorized input" title="vectorized input">

The following function does the vectorization. I am sure that there are much simpler ways to do so, but for the purpose of explanation, this function is implemented in this way.

In [27]:

def matrix_to_vector(input):
    input_h, input_w = input.shape
    output_vector = np.zeros(input_h*input_w, dtype=input.dtype)
    # flip the input matrix up-down because last row should go first
    input = np.flipud(input) 
    for i,row in enumerate(input):
        st = i*input_w
        nd = st + input_w
        output_vector[st:nd] = row
        
    return output_vector


In [28]:
# call the function
vectorized_I = matrix_to_vector(I)
print('vectorized_I: ', vectorized_I)

('vectorized_I: ', array([4, 5, 6, 1, 2, 3]))


## Multiply doubly blocked toeplitz matrix with vectorized input signal
Do the matrix multiplication between these two matrices. In this example, the doubly blocked Toeplitz matrix is $ 12 \times 6$ and the vectorized input is $6 \times 1$ so the result of the multiplication is $12 \times 1$.

In [29]:
# get result of the convolution by matrix mupltiplication
result_vector = np.matmul(doubly_blocked, vectorized_I)
print('result_vector: ', result_vector)

('result_vector: ', array([120., 310., 380., 240.,  70., 230., 330., 240.,  10.,  40.,  70.,
        60.]))


## Last step: reshape the result to a matrix form
From section [output_size](#output-size) we know that the output of the convolution should be $(m_1 + m_2 -1) \times (n_1 + n_2 -1)$. First $(n_1 + n_2 -1)$ elements in the output vector form the last row of the final output and the second $(n_1 + n_2 -1)$ elements go to the second-to-last row of the output matrix. Repeat this process to form the final output matrix.
	
In this example $n_1 = 3$ and $n_2 = 2$, so every 4 element goes to one row of the output matrix.

<img src="images/8.png" alt="reshape the output">

Following function performs this step:

In [30]:

def vector_to_matrix(input, output_shape):
    output_h, output_w = output_shape
    output = np.zeros(output_shape, dtype=input.dtype)
    for i in range(output_h):
        st = i*output_w
        nd = st + output_w
        output[i, :] = input[st:nd]
    # flip the output matrix up-down to get correct result
    output=np.flipud(output)
    return output


In [31]:
# reshape the raw rsult to desired matrix form
out_shape = [output_row_num, output_col_num]
my_output = vector_to_matrix(result_vector, out_shape)

print('Result of implemented method: \n', my_output)

('Result of implemented method: \n', array([[ 10.,  40.,  70.,  60.],
       [ 70., 230., 330., 240.],
       [120., 310., 380., 240.]]))


# Compare the result with other convolution methods
We can compare the output of this method with `convolve2d()` function from the `scipy` library.

In [32]:
from scipy import signal

lib_output = signal.convolve2d(I, F, "full")
print('Result using signal processing library\n', lib_output)

assert(my_output.all() == lib_output.all())

('Result using signal processing library\n', array([[ 10,  40,  70,  60],
       [ 70, 230, 330, 240],
       [120, 310, 380, 240]]))


As you can see the result on the same I and F matrices is same as the result of the implemented method. The parameter "full" is passed to the `signal.convolve2d()` function to get the full convolution results.

# Put everything together

Lets put all the codes in one function and call it with different inputs

In [33]:
import numpy as np
from scipy.linalg import toeplitz


def matrix_to_vector(input):
    """
    Converts the input matrix to a vector by stacking the rows in a specific way explained here
    
    Arg:
    input -- a numpy matrix
    
    Returns:
    ouput_vector -- a column vector with size input.shape[0]*input.shape[1]
    """
    input_h, input_w = input.shape
    output_vector = np.zeros(input_h*input_w, dtype=input.dtype)
    # flip the input matrix up-down because last row should go first
    input = np.flipud(input) 
    for i,row in enumerate(input):
        st = i*input_w
        nd = st + input_w
        output_vector[st:nd] = row   
    return output_vector


def vector_to_matrix(input, output_shape):
    """
    Reshapes the output of the maxtrix multiplication to the shape "output_shape"
    
    Arg:
    input -- a numpy vector
    
    Returns:
    output -- numpy matrix with shape "output_shape"
    """
    output_h, output_w = output_shape
    output = np.zeros(output_shape, dtype=input.dtype)
    for i in range(output_h):
        st = i*output_w
        nd = st + output_w
        output[i, :] = input[st:nd]
    # flip the output matrix up-down to get correct result
    output=np.flipud(output)
    return output


def convolution_as_maultiplication(I, F, print_ir=False):
    """
    Performs 2D convolution between input I and filter F by converting the F to a toeplitz matrix and multiply it
      with vectorizes version of I
      By : AliSaaalehi@gmail.com
      
    Arg:
    I -- 2D numpy matrix
    F -- numpy 2D matrix
    print_ir -- if True, all intermediate resutls will be printed after each step of the algorithms
    
    Returns: 
    output -- 2D numpy matrix, result of convolving I with F
    """
    # number of columns and rows of the input 
    I_row_num, I_col_num = I.shape 

    # number of columns and rows of the filter
    F_row_num, F_col_num = F.shape

    #  calculate the output dimensions
    output_row_num = I_row_num + F_row_num - 1
    output_col_num = I_col_num + F_col_num - 1
    if print_ir: print('output dimension:', output_row_num, output_col_num)

    # zero pad the filter
    F_zero_padded = np.pad(F, ((output_row_num - F_row_num, 0),
                               (0, output_col_num - F_col_num)),
                            'constant', constant_values=0)
    if print_ir: print('F_zero_padded: ', F_zero_padded)

    # use each row of the zero-padded F to creat a toeplitz matrix. 
    #  Number of columns in this matrices are same as numbe of columns of input signal
    toeplitz_list = []
    for i in range(F_zero_padded.shape[0]-1, -1, -1): # iterate from last row to the first row
        c = F_zero_padded[i, :] # i th row of the F 
        r = np.r_[c[0], np.zeros(I_col_num-1)] # first row for the toeplitz fuction should be defined otherwise
                                                            # the result is wrong
        toeplitz_m = toeplitz(c,r) # this function is in scipy.linalg library
        toeplitz_list.append(toeplitz_m)
        if print_ir: print('F '+ str(i)+'\n', toeplitz_m)

        # doubly blocked toeplitz indices: 
    #  this matrix defines which toeplitz matrix from toeplitz_list goes to which part of the doubly blocked
    c = range(1, F_zero_padded.shape[0]+1)
    r = np.r_[c[0], np.zeros(I_row_num-1, dtype=int)]
    doubly_indices = toeplitz(c, r)
    if print_ir: print('doubly indices \n', doubly_indices)

    ## creat doubly blocked matrix with zero values
    toeplitz_shape = toeplitz_list[0].shape # shape of one toeplitz matrix
    h = toeplitz_shape[0]*doubly_indices.shape[0]
    w = toeplitz_shape[1]*doubly_indices.shape[1]
    doubly_blocked_shape = [h, w]
    doubly_blocked = np.zeros(doubly_blocked_shape)

    # tile toeplitz matrices for each row in the doubly blocked matrix
    b_h, b_w = toeplitz_shape # hight and withs of each block
    for i in range(doubly_indices.shape[0]):
        for j in range(doubly_indices.shape[1]):
            start_i = i * b_h
            start_j = j * b_w
            end_i = start_i + b_h
            end_j = start_j + b_w
            doubly_blocked[start_i: end_i, start_j:end_j] = toeplitz_list[doubly_indices[i,j]-1]

    if print_ir: print('doubly_blocked: ', doubly_blocked)

    # convert I to a vector
    vectorized_I = matrix_to_vector(I)
    if print_ir: print('vectorized_I: ', vectorized_I)
    
    # get result of the convolution by matrix mupltiplication
    result_vector = np.matmul(doubly_blocked, vectorized_I)
    if print_ir: print('result_vector: ', result_vector)

    # reshape the raw rsult to desired matrix form
    out_shape = [output_row_num, output_col_num]
    output = vector_to_matrix(result_vector, out_shape)
    if print_ir: print('Result of implemented method: \n', output)
    
    return output

# Test on bigger examples

Now lets test with bigger input and filter matrices and compare out results with results from singla processing function

In [34]:
# test on different examples

# fill I an F with random numbers
I = np.random.randn(10, 13)
F = np.random.randn(30, 70)

my_result = convolution_as_maultiplication(I, F)
print('my result: \n', my_result)
    
from scipy import signal
lib_result = signal.convolve2d(I, F, "full")
print('lib result: \n', lib_result)

assert(my_result.all() == lib_result.all())

('my result: \n', array([[-0.23616414, -0.42490356,  0.63682558, ..., -1.97247008,
        -2.52532425, -1.05906064],
       [ 0.09816711, -1.25735839, -1.83521623, ...,  0.20537352,
         0.10754212, -0.10969996],
       [-0.34553193,  1.80238368,  1.84355807, ...,  0.16008298,
        -4.23089332, -2.75144496],
       ...,
       [ 1.26952309, -2.13368214, -2.10869711, ...,  5.4978565 ,
        -6.92803857, -1.58735277],
       [ 0.18754718,  0.5640235 , -0.64031672, ..., -1.79510593,
         0.44407038, -0.21157057],
       [-0.69797426, -1.11170349, -0.30021619, ...,  1.93949112,
        -5.1205442 ,  0.39370072]]))
('lib result: \n', array([[-0.23616414, -0.42490356,  0.63682558, ..., -1.97247008,
        -2.52532425, -1.05906064],
       [ 0.09816711, -1.25735839, -1.83521623, ...,  0.20537352,
         0.10754212, -0.10969996],
       [-0.34553193,  1.80238368,  1.84355807, ...,  0.16008298,
        -4.23089332, -2.75144496],
       ...,
       [ 1.26952309, -2.13368214, -2.

# To Do
	
 - Add notebook to the project -> Done!
 - Rewrite an efficient code
 - Extend it to handle multi-channel input and filters
 - Make it work with parameters padding='same' or 'valid'
 


 **If you read this post and find any problem or if you have any suggestion, please let me know, thanks.**
 
 # References
 
 - The steps explained here are based on Christophoros Nikou's slides on **Filtering in the Frequency Domain (Circulant Matrices and Convolution)** (http://www.cs.uoi.gr/~cnikou/Courses/Digital_Image_Processing/2010-2011/Chapter_04c_Frequency_Filtering_(Circulant_Matrices).ppt)
 - This post on https://dsp.stackexchange.com/questions/35373/convolution-as-a-doubly-block-circulant-matrix-operating-on-a-vector/35376#35376}{dsp.stackexchange also helped in understanding this algorithm.